In [10]:
import traffic
import numpy as np
import matplotlib.pyplot as plt
import dpkt
import scipy.stats
import scipy.spatial
import copy
import pickle
import socket
import datetime
import timeit
from multiprocessing import Process,Manager
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import os

In [2]:
mtraf= traffic.Traffic('rate_20_30',1005)

In [26]:
mtraf.selectNewSamples(3000)
data=mtraf.getCurrentSamples()

In [28]:

numtrue=10
len_f=900
traindata=[]
trainlabel=[]
okey=data.keys()
for i in xrange(numtrue):
    mtraf.clearCaches()
    d2=mtraf.makeNoisy(0.01,0.01)
    for ind in data:
        
        traindata.append(np.array(data[ind][:len_f]+ d2[ind][:len_f]))
        trainlabel.append(1)
        falseind = ind 
        
        while falseind==ind:
            falseind=np.random.choice(okey)
        traindata.append(np.array(data[ind][:len_f]+ d2[falseind][:len_f]))
        trainlabel.append(0)
            
traindata=np.array(traindata)
trainlabel=np.array(trainlabel)



NameError: name 'data' is not defined

In [2]:
tordata_umass,tordata_dst = pickle.load(open('/home/milad/Research/Watermarking/Code/streamingTest/sites500.pickle','r'))

In [50]:
mlem=99999
for k in okeys:
    mlem=min(mlen,len(tordata_dst[k]),len(tordata_umass[k]))
mlem

300

In [61]:
def slotting(sig):
    stepsize=0.1
    lasttime= stepsize
    curtime=0.0
    for i in sig:
        
        

In [52]:

okeys=tordata_umass.keys()
len_f=300
traindata=[]
trainlabel=[]
for ind in okeys:
        
    
    for _ in range(100):
        traindata.append(np.array(tordata_dst[ind][:len_f]+ tordata_umass[ind][:len_f]))
        trainlabel.append(1)
        falseind = ind 
        while falseind==ind:
            falseind=np.random.choice(okeys)
        traindata.append(np.array(tordata_dst[ind][:len_f]+ tordata_umass[falseind][:len_f]))
        trainlabel.append(0)
traindata=np.array(traindata)
trainlabel=np.array(trainlabel)

In [55]:
per=np.random.permutation(trainlabel.shape[0]-10000)
shuffledtrain=traindata[:-10000,:][per,:]
shuffledtrainlabel=trainlabel[:-10000][per]


In [53]:
print (traindata.shape)

(100200, 600)


In [56]:
print (traindata.shape)
trainsize=shuffledtrainlabel.shape[0]
train_data=shuffledtrain.astype(np.float32)
train_label=shuffledtrainlabel.astype(np.float32)
test_data=traindata[trainsize:].astype(np.float32)
test_label=trainlabel[trainsize:].astype(np.float32)

def acc(predictions,label):
    ll=label.reshape(-1)
    pr=predictions.reshape(-1)
    tp=0
    tt=0
    ff=0
    fp=0
    print (ll.shape)
    for l in xrange(label.shape[0]):
        if ll[l]==1:
            tt+=1
            if pr[l]>0.5:
                tp+=1
        else:
            ff+=1
            if pr[l]>0.5:
                fp+=1
    return tp,tt,fp,ff
            

print (train_data.shape, test_data.shape)

(100200, 600)
(90200, 600) (10000, 600)


In [59]:
print (train_data.shape, test_data.shape)

#print(train_data)
#print(test_label)
print ('DATA LOADED')
batch_size = 128
beta = 0.000

h1_nodes = 500
h2_nodes = 200
h3_nodes = 200

graph = tf.Graph()

with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, 600))
    tf_train_label = tf.placeholder(tf.float32, shape=(batch_size, 1))
    tf_test_dataset = tf.constant(test_data)
    
    #conv1= tf.nn.conv2d(tf_train_dataset,filtees=32, kernel_size=[2,2],padding="same")
    
    # H1
    h1_w = tf.Variable(tf.truncated_normal([600, h1_nodes]))
    h1_b = tf.Variable(tf.truncated_normal([h1_nodes]))
    h1 = tf.nn.sigmoid(tf.matmul(tf_train_dataset, h1_w) + h1_b)

    # H2
#     h2_w = tf.Variable(tf.truncated_normal([h1_nodes, h2_nodes]))
#     h2_b = tf.Variable(tf.truncated_normal([h2_nodes]))
#     h2 = tf.nn.sigmoid(tf.matmul(h1, h2_w) + h2_b)

#     #H3
#     h3_w = tf.Variable(tf.truncated_normal([h2_nodes, h3_nodes]))
#     h3_b = tf.Variable(tf.truncated_normal([h3_nodes]))
#     h3 = tf.nn.sigmoid(tf.matmul(h2, h3_w) + h3_b)

    # last

    w = tf.Variable(tf.truncated_normal([h1_nodes, 1]))
    b = tf.Variable(tf.truncated_normal([1]))

    logits = tf.matmul(h1,w)+b
    wloss=tf.nn.l2_loss(w)
#    ratio = 0.001
#    class_weight = tf.constant([ratio, 1.0 - ratio])

#    weighted_logits = tf.mul(logits, class_weight) # shape [batch_size, 2]


    loss= tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits,tf_train_label) + beta * wloss,name="xent_raw")
    global_step = tf.Variable(0, trainable=False)
    starter_learning_rate = 0.9
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           100000, 0.96, staircase=True)

    optimizer= tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)

    prediction = tf.nn.sigmoid(logits)
    test_relu1= tf.nn.sigmoid(tf.matmul(tf_test_dataset,h1_w)+h1_b)
    #test_relu2= tf.nn.sigmoid(tf.matmul(test_relu1,h2_w)+h2_b)
#     test_relu3= tf.nn.sigmoid(tf.matmul(test_relu2,h3_w)+h3_b)
    test_prediction= tf.nn.sigmoid(tf.matmul(test_relu1,w)+b)


(90200, 600) (10000, 600)
DATA LOADED


In [60]:

numstep = 20001
with tf.Session(graph=graph) as ses:
    tf.initialize_all_variables().run()
    print ('TF INIT')
    for step in range(numstep):
        offset = (step*batch_size) % (train_label.shape[0]-batch_size)
        batch_data=train_data[offset:offset+batch_size,:]
        batch_label=train_label[offset:offset+batch_size].reshape(-1,1)

        feed_dict= {tf_train_dataset:batch_data, tf_train_label:batch_label,}
        _,l,lw,pp = ses.run([optimizer,loss,wloss,prediction],feed_dict=feed_dict)
        if step % 1000==0:
        
            print ("batch  %d ,loss:%f, wloss %f "%(step,l,lw,),(acc(pp,batch_label)))
            tt=test_prediction.eval()
            #print (train_label.reshape(-1,1)[:10])
            #print ((np.array(tt[:10])+0.5).astype(np.int32))
            print ("Test ACC",acc(tt,test_label))


TF INIT
(128,)
batch  0 ,loss:3.064606, wloss 195.352875  (53, 66, 56, 62)
(10000,)
Test ACC (0, 5000, 0, 5000)
(128,)
batch  1000 ,loss:0.626413, wloss 72.032600  (8, 57, 0, 71)
(10000,)
Test ACC (3400, 5000, 2783, 5000)
(128,)
batch  2000 ,loss:0.326151, wloss 165.882385  (67, 67, 15, 61)
(10000,)
Test ACC (2400, 5000, 1072, 5000)
(128,)
batch  3000 ,loss:0.222131, wloss 268.775421  (69, 69, 13, 59)
(10000,)
Test ACC (2000, 5000, 757, 5000)
(128,)
batch  4000 ,loss:0.233952, wloss 369.211304  (64, 66, 4, 62)
(10000,)
Test ACC (2800, 5000, 1143, 5000)
(128,)
batch  5000 ,loss:0.158439, wloss 464.130249  (64, 64, 8, 64)
(10000,)
Test ACC (1000, 5000, 358, 5000)
(128,)
batch  6000 ,loss:0.106483, wloss 552.308594  (61, 61, 4, 67)
(10000,)
Test ACC (1100, 5000, 403, 5000)
(128,)
batch  7000 ,loss:0.147349, wloss 633.736206  (62, 62, 2, 66)
(10000,)
Test ACC (1200, 5000, 345, 5000)
(128,)
batch  8000 ,loss:0.068204, wloss 707.838013  (64, 64, 4, 64)
(10000,)
Test ACC (900, 5000, 296, 5000

In [2]:
import numpy as np

def calc_MI(X,Y,bins):

   c_XY = np.histogram2d(X,Y,bins)[0]
   c_X = np.histogram(X,bins)[0]
   c_Y = np.histogram(Y,bins)[0]

   H_X = shan_entropy(c_X)
   H_Y = shan_entropy(c_Y)
   H_XY = shan_entropy(c_XY)

   MI = H_X + H_Y - H_XY
   return MI

def shan_entropy(c):
    c_normalized = c / float(np.sum(c))
    c_normalized = c_normalized[np.nonzero(c_normalized)]
    H = -sum(c_normalized* np.log2(c_normalized))  
    return H

A = np.array([[ 2.0,  140.0,  128.23, -150.5, -5.4  ],
              [ 2.4,  153.11, 130.34, -130.1, -9.5  ],
              [ 1.2,  156.9,  120.11, -110.45,-1.12 ]])

bins = 5 # ?
n = A.shape[1]
matMI = np.zeros((n, n))

for ix in np.arange(n):
    for jx in np.arange(ix+1,n):
        matMI[ix,jx] = calc_MI(A[:,ix], A[:,jx], bins)

In [8]:
calc_MI(np.arange(5),np.arange(5,10),4)

1.9219280948873623

In [9]:
np.histogram2d(np.arange(5),np.arange(5,10),bins)[0]

array([[ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.]])